In [ ]:
# ==========================================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt #数据可视化
import numpy as np 
from PIL import Image
from torchvision import datasets, transforms
from torchsummary import summary #从 torchsummary里导入 summary，打印显示网络结构和参数，


# torchvision是独立于pytorch的关于图像操作的一些方便工具库。
# vision.datasets : 几个常用视觉数据集，可以下载和加载
# vision.models : 流行的模型，例如 AlexNet, VGG, ResNet 和 Densenet 以及训练好的参数。
# vision.transforms : 常用的图像操作，例如：数据类型转换，图像到tensor ,numpy 数组到tensor , tensor 到 图像等。
# vision.utils : 用于把形似 (3 x H x W) 的张量保存到硬盘中，给一个mini-batch的图像可以产生一个图像格网

'''
torchvision是PyTorch中专门用来处理图像的库，这个包中常用的几个模块：

torchvision.datasets：是用来进行数据加载的

torchvision.models：为我们提供了已经训练好的模型，让我们加载之后可以直接使用。包括AlexNet、VGG、ResNet

torchvision.transforms：为我们提供了一般的图像转换操作类。

torchvision.utils:将给定的Tensor保存成image文件。
'''

#==========================================================================================
# 设置超参数

BATCH_SIZE = 512      # 每批训练的样本数量
EPOCHS = 20            # 总的训练回合


'''
batch的意思是“批”，即把一定数目的Sample组合到一起，组成一个batch。
batch_size表示一个batch中Sample的个数.

批量梯度下降 —> batch_size=训练集的大小（整个训练集作为一个batch）
随机梯度下降 —> batch_size= 1（每个batch中只有一个Sample）
小批量梯度下降 —> 1 <batch_size<训练集的大小(每个batch中Sample的个数大于1且小于训练集Sample的总数)

'''
#===========================================================================================

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 选择运行设备 cuda or cpu
print('GPU是否可用：', torch.cuda.is_available())

#============================================================================================
# 下载训练集
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train = True, download = True,
              transform = transforms.Compose([               #  将多个transform组合起来使用
                  transforms.ToTensor(),                     # 将图像转换为tensor对象  
                  transforms.Normalize((0.1037,), (0.3081,)) # 标准化
              ])),
    batch_size = BATCH_SIZE, shuffle = True) 

# transforms.Normalize((0.1037,), (0.3081,)) 服从正态分布 0.1037和 0.3081是 mnist数据集的均值和标准差

# shuffle = True表示随机抽取 batch_size个样本作为一个批次返回
# shuffle() 方法将序列的所有元素随机排序

'''
transforms.Normalize(mean,std,inplace=False):
功能:逐channel的对图像进行标准化(数据标准化可以加速数据的收敛), 
      将会把tensor正则化，即 Normalized_image=(image-mean)/std

transforms.Normalize使用如下公式进行归一化：
channel=（channel-mean）/std

mean：各通道的均值;
std：各通道的标准差;
inplace：是否原地操作

'''


# vision.utils : 用于把形似 (3 x H x W) 的张量保存到硬盘中，给一个mini-batch的图像可以产生一个图像格网
#------------------------------------------------------------------------------------------
# 测试集
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)            # shuffle() 方法将序列的所有元素随机排序

# 打印训练集和测试集的长度
print('len(train_loader)={}'.format(len(train_loader))) 
print('len(train_loader)={}'.format(len(test_loader))) 


# 显示一张图片
def imshow(img):
    img = img / 2 + 0.5      # 逆归一化
    npimg = img.numpy()      # 将张量转换成numpy数组
    plt.imshow(np.transpose(npimg, (1, 2, 0))) # transpose交换维度（0，1，2）转换为（1，2，0） imshow画图
    plt.show()               # 显示图片
    
#
# 得到一些随机训练图像
dataiter = iter(train_loader) # 生成迭代器 
images, labels = dataiter.next()

# 显示图像
imshow(torchvision.utils.make_grid(images))

print(images.shape)# 显示 batch_size,Channels,Width,Heigght
#=========================================================================================
# 用类定义模型
# 下面我们定义一个网络，网络包含两个卷积层，conv1和conv2，
# 然后紧接着两个线性层作为输出，
# 最后输出10个维度，这10个维度我们作为0-9的标识来确定识别出的是那个数字
#------------------------------------------------------------------------
# 用 nn.Module创建一个 convNet类,以类的方法来创建网络
class ConvNet(nn.Module):
    def __init__(self): # 定义构造方法函数，用来实例化
        super().__init__()
        #1*1*28*28
        self.conv1 = nn.Conv2d(1, 10, 5) 
        # 第一个卷积层：输入通道数=1，输出通道数=10，卷积核大小=5*5，默认步长=1
        
        self.conv2 = nn.Conv2d(10, 20, 3)
        # 第二个卷积层：输入通道数=10，输出通道数=20，卷积核大小=3*3，默认步长=1
        
        self.fc1 = nn.Linear(20 * 10 * 10, 500)
        # 第一个全连接层：输入特征数=2000，输出特征数=500
        # 也可以理解成：将20*10*10个节点连接到500个节点上
        
        self.fc2 = nn.Linear(500, 10)
        # 第二个全连接层：输入特征数=500，输出特征数=10（这10个维度我们作为0-9的标识来确定识别出的是那个数字。）
        # 也可以理解成：将500个节点连接到10个节点上
#------------------------------------------------------------------------
    def forward(self, x):#定义前向计算函数
        in_size = x.size(0) #  # in_size 为 batch_size（一个batch中的Sample数）  1 * 1 * 28 * 28
        out= self.conv1(x)  # x经过第一个卷积层得到一个输出    大小是：1* 10 * 24 *24
        out = F.relu(out)   # 对上一个输出进行relu激活更新out  大小是：1 * 10 * 24 * 24
        out = F.max_pool2d(out, 2, 2) # 对out作最大池化更新out 大小是：   1* 10 * 12 * 12
        out = self.conv2(out)         # 对out作卷积处理更新out 大小是：1* 20 * 10 * 10
        out = F.relu(out)             # 对out进行激活  大小是：1 * 20 * 10 * 10
        out = out.view(in_size, -1)   # .view()函数作用是将一个多行的Tensor转化成一维
        out = self.fc1(out)           # 全连接层 1 * 500
        out = F.relu(out)             # 激活处理 1 * 500
        out = self.fc2(out)           # 全连接层 1 * 10
        out = F.log_softmax(out, dim = 1) # 输出概率最大的out  大小是 1 *10
        # F.log_softmax（）将数据的范围改到[0, 1]之内，表示概率。
        return out
#=============================================================================================
'''
------------------------------------------------------
输出公式 =（M-K+2P）/S +1
M：输入神经元个数/大小
K：卷积大小
P：零填充
S：步长

原始图像大小：28*28*1
------------------------------------------------------
第一个卷积层:self.conv1=nn.Conv2d(1,10,5) ：
其参数意义为：
    输入通道为 1 （输入图像是灰度图）
    输出通道为 10（10分类问题，所以需要用到的卷积核就有 10 种）
    卷积核 kernel_size为 5×5

    24输出维度 = 28输入维度 - 5卷积核size + 1
    所以输出 shape 为：10 × 24 × 24
-----------------------------------------------------
第一个激活函数：out = F.relu(out)
输出维度不变仍为 10 × 24 × 24
-----------------------------------------------------
第一个最大池化层：out = F.max_pool2d(out, 2, 2)
    该最大池化层在 2x2 空间里向下采样。
    12输出维度 = 24输入维度 / 2。
    所以输出 shape 为：10 × 12 × 12
-----------------------------------------------------
第二个卷积层self.conv2=nn.Conv2d(10, 20,3)
其参数意义为：
    输入通道为 10 （第一个最大池化层的输出通道数）
    输出通道为 20 （需要用到的卷积核就有 20 种）
    卷积核kernel_size为 3×3
    10输出维度 = 12输入维度 - 3卷积核size + 1
    所以输出 shape 为：20 × 10 × 10
----------------------------------------------------
第二个激活函数out = F.relu(out)
特征提取结束

输出前的数据预处理
因为全连接层Linear的输出为最后的输出，
而全连接层Linear要求的输入为展平后的多维的卷积成的特征图（特征图为特征提取部分的结果）

输出前的数据预处理结束
-----------------------------------------------------
输出即全连接层

第一个全连接层self.fc1=nn.Linear(20*10*10, 500)
输入维度为 20 * 10 * 10= 2000
设定的输出维度为 500 × 1
-----------------------------------------------------
第三个激活函数out = F.relu(out)
输出维度不变，仍为 500 × 1
-----------------------------------------------------
第二个全连接层self.fc2=nn.Linear(500, 10)
输入维度为 500 × 1
输出维度设定为 10 × 1（因为是一个10分类的问题，所以最后要变成 10 × 1）
-----------------------------------------------------
第三个激活函数out = F.log_softmax(out, dim=1)
用F.log_softmax()将数据的范围改到[0, 1]之内，表示概率。
输出维度仍为 10 × 1，其值可以视为概率。

'''
# ================================================================================    
# 实例化一个网络，实例化后使用“.to”方法将网络移动到 GPU
model = ConvNet().to(DEVICE) # 将模型转移到cuda上
print(model)                 # 打印模型
# summary(model, input_size=(1, 28, 28))#打印显示网络结构和参数，
#=================================================================================
# 定义优化器
optimizer = optim.Adam(model.parameters())
# 损失函数为 负对数似然函数
# Negative Log Likelihood(NLL) Loss:负对数似然损失函数
#=================================================================================
# 定义训练函数，将训练的所有操作都封装到 train函数中
def train(model, device, train_loader, optimizer, epoch):
    model.train() # 作用是启用 batch_normalization 和 drop out
    for batch_idx, (data, target) in enumerate(train_loader): # 循环次数为 batch_idx
        data, target = data.to(device), target.to(device)     # 将数据转移到cuda上
        optimizer.zero_grad() # 梯度清零
        output = model(data)  # 获得模型输出
        loss = F.nll_loss(output,target) # 计算输出与目标之间的损失
        loss.backward()       # 反向计算梯度
        optimizer.step()      # 优化更新参数
        if (batch_idx + 1) % 30 == 0:    # 每隔 30 mini_batch输出一次
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
'''
model指的是实例化网络

enumerate()函数将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标，一般用在 for 循环当中。

batch_idx的最大循环值 +1 = (MNIST数据集样本总数60000/ BATCH_SIZE 100)
每 30 个小批次输出一次，一直输出到 batch_idx所允许的最大值------> batch_idx每次增加30一直增加到所允许的最大值
(batch_idx + 1）是30的整数倍


model.train()在训练过程中使用，在使用pytorch构建神经网络的时候，，作用是启用 batch_normalization 和 drop out，
model.train()将模型转化为训练模式。

model.eval()在测试过程中使用，这时神经网络会沿用batch_normalization的值，但是不会使用 drop out。

通过 batch_idx 来获取数据下标，
用 data  获得待处理的数据（由手写数字的image转成的，被标准化的tensor，详见训练数据集下载的part），
用 target获取实际值；最后由for循环遍历整个train_loader

'''            
#====================================================================================                     
# 定义测试函数,将测试的所有操作都封装到 test 函数中
def test(model, device, test_loader):
    model.eval() # 测试之前必须执行  model.eval()
    test_loss =0 # 测试集的损失初始为0
    correct = 0  # 测试集的正确率初始为0
    with torch.no_grad(): # 此时已经不需要计算梯度，也不会进行反向传播
        for data, target in test_loader: # 将 data 和 target 在测试加载器上进行迭代
            data, target = data.to(device), target.to(device) # 将数据转移到cuda上
            output = model(data)                              # 由 model 获得模型的输出
            test_loss += F.nll_loss(output, target, reduction = 'sum') # 测试集损失累加 将一批的损失相加
            pred = output.max(1, keepdim = True)[1] # 将输出结果概率最大的作为预测值，找到概率最大的下标,输出最大值的索引位置
            correct += pred.eq(target.view_as(pred)).sum().item() # 正确率累加
    
    test_loss /= len(test_loader.dataset) # 测试集平均损失
    print("\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%) \n".format(
        test_loss, correct, len(test_loader.dataset),
        100.* correct / len(test_loader.dataset)
            ))
#====================================================================================    
# 最后开始训练和测试
for epoch in range(1,EPOCHS+1):
    train(model,  DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)